In [1]:
import pandas as pd
import numpy as np

import ibm_db
import ibm_db_dbi
import ibm_db_sa

import datetime as dt
import time
from datetime import date
from dateutil import relativedelta

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import os
from tqdm import tqdm_notebook
import sqlalchemy
from functools import reduce
import pickle
import joblib
import io

import os
from project_lib import Project
project = Project()
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [2]:
dsn = "DRIVER={IBM DB2 ODBC DRIVER};DATABASE=bludb;HOSTNAME=10.1.50.25;PORT=50000;PROTOCOL=TCPIP;UID=tpbdatait1;PWD=Tpbdatait1@123;"
hdbc = ibm_db.connect(dsn, "", "")
hdbi = ibm_db_dbi.Connection(hdbc)

pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(suppress= True)

In [57]:
def clean_whitelist_xlsx_file(df_whitelist):
    if list(df_whitelist.columns)[1] == 'Số CIF':
        df_whitelist.rename(columns={'Số CIF':'cif_number', 'Kỳ cập nhật':'obs_date'}, inplace=True)
    else:
        df_whitelist.rename(columns={'CIF_whitelist':'cif_number', 'Ob_date_whitelist':'obs_date'}, inplace=True)
    df_whitelist.cif_number = df_whitelist.cif_number.astype(str).str.zfill(8)
    df_whitelist.obs_date = pd.to_datetime(df_whitelist.obs_date, format='%d/%m/%Y')
    return df_whitelist

def read_whitelist_file(whitelist_file):
    df_whitelist = pd.read_excel(wslib.mount.get_data_path(whitelist_file), sheet_name=0)
    df_whitelist = clean_whitelist_xlsx_file(df_whitelist)
    return df_whitelist

def create_label(cashloan_file, whitelist_file):
    df_applied_cashloan = pd.read_excel(wslib.mount.get_data_path(cashloan_file), sheet_name='SLP Report')
    df_whitelist = read_whitelist_file(whitelist_file)
    
    df_applied_cif = pd.DataFrame(data={'cif_number':df_applied_cashloan['CASA account number'].str[:8].str.zfill(8).unique()
                                        , 'label':'1'})
    df_applied_merge = df_applied_cif.merge(df_whitelist, on='cif_number', how='right')
    df_label_1 = df_applied_merge[~df_applied_merge['label'].isnull()][['cif_number', 'label', 'obs_date']]
    
    df_label_0 = df_whitelist[~df_whitelist.cif_number.isin(df_label_1.cif_number.unique())][['cif_number', 'obs_date']]
    df_label_0 = df_label_0[df_label_0.obs_date != pd.to_datetime('2022-08-30')]
    df_label_0['label'] = '0'
    
    df_label = pd.concat([df_label_1, df_label_0])
    return df_label

## OLD A_SCORE (old_a)

In [58]:
df_label_old_a = create_label('Dico (update tới hết 23.9).xlsx', 'whitelist.XLSX')

In [61]:
filename_old_a = 'df_label_old_a_221206.pkl'
buffer_old_a = pickle.dumps(df_label_old_a)
wslib.save_data(filename_old_a, buffer_old_a, overwrite = True)

{'name': 'df_label_old_a_221206.pkl',
 'asset_type': 'data_asset',
 'asset_id': '1facec06-ec44-4d9b-a41f-b901411e12a2',
 'attachment_id': '98203bf6-b4ed-4a55-b7f1-68c23f70f16b',
 'filepath': 'df_label_old_a_221206.pkl',
 'data_size': None,
 'mime': 'application/binary',
 'summary': ['created or overwritten file',
  'created data asset',
  'created attachment']}